# Exploratory Data Analysis

## Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Load data

In [4]:
train = pd.read_csv('data/no1_train.csv')
test = pd.read_csv('data/no1_validation.csv')
train.head()

,start_time,hydro,micro,thermal,wind,river,total,y,sys_reg,flow
0,2019-01-09 14:10:00,1591.839,141.928,11.5,74.428,0.0,1819.695,341.330021,10.0,-4417.4
1,2019-01-09 14:15:00,1591.839,141.928,11.5,74.428,0.0,1819.695,330.114330,10.0,-4417.4
2,2019-01-09 14:20:00,1591.839,141.928,11.5,74.428,0.0,1819.695,323.877221,10.0,-4417.4
3,2019-01-09 14:25:00,1591.839,141.928,11.5,74.428,0.0,1819.695,296.438463,10.0,-4417.4
4,2019-01-09 14:30:00,1591.839,141.928,11.5,74.428,0.0,1819.695,269.097132,10.0,-4417.4


In [11]:
train.describe()

,hydro,micro,thermal,wind,river,total,y,sys_reg,flow
count,225088.000000,225088.000000,225088.000000,225088.000000,225088.0,225088.000000,225088.000000,225088.000000,225088.000000
mean,1888.334127,224.189826,21.803497,55.703266,0.0,2190.031963,8.907921,-8.284305,-1999.055081
std,401.000305,58.559855,3.579229,47.053033,0.0,444.077730,320.262539,43.748315,1311.682286
min,683.438000,80.371000,0.000000,0.000000,0.0,849.732000,-1579.680903,-828.000000,-5541.200000
25%,1625.029000,185.017000,22.100000,15.704000,0.0,1916.523000,-142.226700,0.000000,-2996.700000
50%,1933.916000,225.938000,22.100000,41.848000,0.0,2248.874000,-11.282351,0.000000,-1953.700000
75%,2175.595000,269.558000,24.100000,88.723000,0.0,2497.792000,127.454943,0.000000,-906.900000
max,2995.524000,349.271000,25.700000,176.000000,0.0,3351.974000,2956.333317,474.000000,723.400000
